In [1]:
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize
import os 
os.environ['OMP_NUM_THREADS'] = '8'
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.cross_validation import train_test_split
TRAIN_SAMPLE_SIZE = 800000
from datetime import time
from scipy import stats

/home/hnpathak/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/hnpathak/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
morning_start = time(6)
morning_end = time(12)
afternoon_start = time(13)
afternoon_end = time(17)
evening_start = time(18)
evening_end = time(22)
night_start= time(23)
night_end= time(5)

In [3]:
final_test = pd.read_csv('./data/test/test.csv')
dff = final_test
dff.iloc[:,6] = pd.to_datetime(dff.iloc[:,6])
dff['time_of_day'] = dff.iloc[:,6].apply(lambda x: x.time())
#df['time_of_day'].head
dff['morning'] = (dff['time_of_day'] >= time(8,0)) & (dff['time_of_day'] <= time(12,0))
dff['afternoon'] = (dff['time_of_day'] >= time(12,1)) & (dff['time_of_day'] <= time(17,0))
dff['evening'] = (dff['time_of_day'] >= time(17,1)) & (dff['time_of_day'] <= time(20,0))
dff['night'] = (dff['time_of_day'] >= time(20,1)) & (dff['time_of_day'] <= time(23,59))
dff['late_night'] = (dff['time_of_day'] >= time(0,1)) & (dff['time_of_day'] <= time(8,0))
dff['night'] = dff['night'].astype(float)
dff['afternoon'] = dff['afternoon'].astype(float)
dff['late_night'] = dff['late_night'].astype(float)
dff['morning'] = dff['morning'].astype(float)
dff['evening'] = dff['evening'].astype(float)
test_df = dff
#del df
test_df.head()

,click_id,ip,app,device,os,channel,click_time,time_of_day,morning,afternoon,evening,night,late_night
0,0,5744,9,1,3,107,2017-11-10 04:00:00,04:00:00,0.0,0.0,0.0,0.0,1.0
1,1,119901,9,1,3,466,2017-11-10 04:00:00,04:00:00,0.0,0.0,0.0,0.0,1.0
2,2,72287,21,1,19,128,2017-11-10 04:00:00,04:00:00,0.0,0.0,0.0,0.0,1.0
3,3,78477,15,1,13,111,2017-11-10 04:00:00,04:00:00,0.0,0.0,0.0,0.0,1.0
4,4,123080,12,1,13,328,2017-11-10 04:00:00,04:00:00,0.0,0.0,0.0,0.0,1.0


In [4]:
test_df2 = test_df.iloc[:,[1,2,3,4,5,8,9,10,11,12]]
test_df2.head()

,ip,app,device,os,channel,morning,afternoon,evening,night,late_night
0,5744,9,1,3,107,0.0,0.0,0.0,0.0,1.0
1,119901,9,1,3,466,0.0,0.0,0.0,0.0,1.0
2,72287,21,1,19,128,0.0,0.0,0.0,0.0,1.0
3,78477,15,1,13,111,0.0,0.0,0.0,0.0,1.0
4,123080,12,1,13,328,0.0,0.0,0.0,0.0,1.0


In [5]:
del dff, test_df

In [49]:
Y_LABEL = 'y'                                   			        # Name of the variable to be predicted
#KEYS = [i for i in raw_data.keys().tolist() if i != Y_LABEL]	# Name of predictors
                      			    # Input size
N_CLASSES = 2                                    			    # Number of classes (output size)
LEARNING_RATE = 0.001                               			    # Learning rate
TRAINING_EPOCHS = 10                             			    # Number of epochs
BATCH_SIZE = 1500                                    			    # Batch size
DISPLAY_STEP = 4                                    			    # Display progress each x epochs
HIDDEN_SIZE = 10          	                                   		      # Number of hidden neurons 256
ACTIVATION_FUNCTION_OUT = tf.nn.sigmoid                          # Last layer act fct
STDDEV = 0.1                                        			    # Standard deviation (for weights random init)
RANDOM_STATE = 100

n_input = 10                   # input n labels
n_hidden_1 = 10            # 1st layer
n_hidden_2 = 20            # 2nd layer
n_hidden_3 = 20            # 3rd layer
n_hidden_4 = 20           # 4th layer
n_hidden_5 = 10 
n_classes = N_CLASSES               # output m classes

In [50]:
X = tf.placeholder(tf.float32, [None, n_input],name='X')
y = tf.placeholder(tf.float32, [None, n_classes],name='y')
dropout_keep_prob = tf.placeholder(tf.float32)


weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1],stddev=STDDEV)),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2],stddev=STDDEV)),
    'h3': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_3],stddev=STDDEV)),
    'h4': tf.Variable(tf.random_normal([n_hidden_3, n_hidden_4],stddev=STDDEV)),
    'h5': tf.Variable(tf.random_normal([n_hidden_4, n_hidden_5],stddev=STDDEV)),
    'out': tf.Variable(tf.random_normal([n_hidden_5, n_classes],stddev=STDDEV)),                                   
}

biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'b3': tf.Variable(tf.random_normal([n_hidden_3])),
    'b4': tf.Variable(tf.random_normal([n_hidden_4])),
    'b5': tf.Variable(tf.random_normal([n_hidden_5])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}    
   
def mlp(_X, _weights, _biases, dropout_keep_prob):
    layer1 = tf.nn.dropout(tf.nn.relu(tf.add(tf.matmul(_X, _weights['h1']), _biases['b1'])), dropout_keep_prob)
    layer2 = tf.nn.dropout(tf.nn.relu(tf.add(tf.matmul(layer1, _weights['h2']), _biases['b2'])), dropout_keep_prob)
    layer3 = tf.nn.dropout(tf.nn.relu(tf.add(tf.matmul(layer2, _weights['h3']), _biases['b3'])), dropout_keep_prob)
    layer4 = tf.nn.dropout(tf.nn.relu(tf.add(tf.matmul(layer3, _weights['h4']), _biases['b4'])), dropout_keep_prob)
    layer5 = tf.nn.dropout(tf.nn.relu(tf.add(tf.matmul(layer4, _weights['h5']), _biases['b5'])), dropout_keep_prob)
    out = ACTIVATION_FUNCTION_OUT(tf.add(tf.matmul(layer5, _weights['out']), _biases['out']))
    return out

In [51]:
pred = mlp(X, weights, biases, dropout_keep_prob)

# Loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y)) # softmax loss
optimizer = tf.train.AdamOptimizer(learning_rate = LEARNING_RATE).minimize(cost)

# Accuracy
correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
confusion_matrix = tf.confusion_matrix(labels= tf.argmax(y, 1),predictions = tf.argmax(pred, 1), num_classes=2)
predictions =  tf.argmax(pred,1)

In [53]:
data_pos = pd.read_csv('data/positives/positives.csv',header=None)
print('Negative samples')
items = os.listdir('./data/negatives/')
#count = 0
for item in items:
    #NEGATIVE data
    data_neg = pd.read_csv('data/negatives/'+str(item), header=None)
    print(data_neg.shape)
    #Combine one section of positive and negative
    data_combined =  pd.concat([data_pos, data_neg])
    data_combined = data_combined.sample(frac=1)
    print('data_combined: ', data_combined.shape)
    #keep interested columns
    data_f = data_combined.iloc[:,0:6]
    #data_f.astype(float)
    labels = data_combined.iloc[:,7:8]
    labels.astype(float)
    #n_values = np.max(labels) 
    labels_ = np.eye(2)[(labels.values).reshape(-1)]
    #convert the date and time
    df = data_f
    df.iloc[:,5] = pd.to_datetime(df.iloc[:,5])
    df['time_of_day'] = df.iloc[:,5].apply(lambda x: x.time())
    #df['time_of_day'].head
    df['morning'] = (df['time_of_day'] >= time(8,0)) & (df['time_of_day'] <= time(12,0))
    df['afternoon'] = (df['time_of_day'] >= time(12,1)) & (df['time_of_day'] <= time(17,0))
    df['evening'] = (df['time_of_day'] >= time(17,1)) & (df['time_of_day'] <= time(20,0))
    df['night'] = (df['time_of_day'] >= time(20,1)) & (df['time_of_day'] <= time(23,59))
    df['late_night'] = (df['time_of_day'] >= time(0,1)) & (df['time_of_day'] <= time(8,0))
    df['night'] = df['night'].astype(float)
    df['afternoon'] = df['afternoon'].astype(float)
    df['late_night'] = df['late_night'].astype(float)
    df['morning'] = df['morning'].astype(float)
    df['evening'] = df['evening'].astype(float)
    ###########################
    df2 = df.iloc[:,[0,1,2,3,4,7,8,9,10,11]]
    df2.head()
    df2.astype(float).head()
    #split the data set
    
    x_train, x_test, y_train, y_test = train_test_split(df2,labels_,test_size = 0.1,random_state = 100)
    x_train.astype(float)
    x_test.astype(float)
    y_train.astype(float)
    y_test.astype(float)
    x_train = stats.zscore(x_train, axis=1)
    x_test = stats.zscore(x_test, axis=1)
    test_df2_zvalues = stats.zscore(test_df2.values, axis=1)
    ##################################
    ##################################
    N_INSTANCES = df2.shape[0]
    
    ###################################
    #tf.set_random_seed(1234)
    #saver = tf.train.Saver()
    with tf.Session() as sess:
        tf.global_variables_initializer().run()

        for epoch in range(TRAINING_EPOCHS):
            avg_cost = 0.
            total_batch = int(x_train.shape[0] / BATCH_SIZE)
            X_batches = np.array_split(x_train, total_batch)
            Y_batches = np.array_split(y_train, total_batch)
            # Loop over all batches
            for i in range(total_batch):
                batch_xs, batch_ys =  X_batches[i], Y_batches[i]
                # Fit using batched data
                sess.run(optimizer, feed_dict={X: batch_xs, y: batch_ys, dropout_keep_prob: 0.85})
                # Calculate average cost
                avg_cost += sess.run(cost, feed_dict={X: batch_xs, y: batch_ys, dropout_keep_prob:1.})/total_batch
            # Display progress
            if epoch % DISPLAY_STEP == 0:
                print("Epoch: %03d/%03d cost: %.9f" % (epoch, TRAINING_EPOCHS, avg_cost))
                train_acc = sess.run(accuracy, feed_dict={X: batch_xs, y: batch_ys, dropout_keep_prob:1.})
                print("Training accuracy: %.3f" % (train_acc))
        test_acc = sess.run(accuracy, feed_dict={X: x_test, y: y_test, dropout_keep_prob:1.})
        confusion = sess.run(confusion_matrix, feed_dict={X: x_test, y: y_test, dropout_keep_prob:1.})
        print("Test accuracy: %.3f" % (test_acc))
        answer  = sess.run(predictions, feed_dict={X: x_test, y: y_test, dropout_keep_prob:1.})
        final_test_acc = sess.run(predictions, feed_dict={X: test_df2_zvalues, y: y_test, dropout_keep_prob:1.})
        #print(answer)
                
        final_test_acc = final_test_acc.astype(np.int32)
        final_test['click_id'] = final_test['click_id'].astype(np.int32)
        a = np.vstack((final_test['click_id'].values,final_test_acc))
        a = a.T
        a = np.int32(a)

        np.savetxt('./m1/final_test_'+item+'.csv',a, fmt='%i', delimiter=",")
        #np.savetxt('./m1/answer', answer)
        print(item)
        print(confusion)
        #saver.save(sess, './m1/model/')


Negative samples
(736577, 8)
data_combined:  (1193423, 8)
Epoch: 000/010 cost: 0.665763421
Training accuracy: 0.630
Epoch: 004/010 cost: 0.567099545
Training accuracy: 0.733
Epoch: 008/010 cost: 0.562272491
Training accuracy: 0.724
Test accuracy: 0.734
xbu
[[58998 14636]
 [17056 28653]]
(738504, 8)
data_combined:  (1195350, 8)
Epoch: 000/010 cost: 0.665600180
Training accuracy: 0.616
Epoch: 004/010 cost: 0.593975181
Training accuracy: 0.696
Epoch: 008/010 cost: 0.589958001
Training accuracy: 0.713
Test accuracy: 0.710
xjc
[[60132 13694]
 [21017 24692]]
(738286, 8)
data_combined:  (1195132, 8)
Epoch: 000/010 cost: 0.665679835
Training accuracy: 0.617
Epoch: 004/010 cost: 0.585954251
Training accuracy: 0.731
Epoch: 008/010 cost: 0.585078509
Training accuracy: 0.715
Test accuracy: 0.726
xgl
[[61267 12644]
 [20149 25454]]
(737967, 8)
data_combined:  (1194813, 8)
Epoch: 000/010 cost: 0.665744399
Training accuracy: 0.606
Epoch: 004/010 cost: 0.564140387
Training accuracy: 0.728
Epoch: 008/01

Epoch: 004/010 cost: 0.569123610
Training accuracy: 0.743
Epoch: 008/010 cost: 0.567225293
Training accuracy: 0.746
Test accuracy: 0.708
xfc
[[52993 20536]
 [14328 31607]]
(736890, 8)
data_combined:  (1193736, 8)
Epoch: 000/010 cost: 0.665863642
Training accuracy: 0.634
Epoch: 004/010 cost: 0.564865261
Training accuracy: 0.769
Epoch: 008/010 cost: 0.560720078
Training accuracy: 0.725
Test accuracy: 0.745
xbc
[[68117  5663]
 [24761 20833]]
(738817, 8)
data_combined:  (1195663, 8)
Epoch: 000/010 cost: 0.665411417
Training accuracy: 0.631
Epoch: 004/010 cost: 0.580118318
Training accuracy: 0.729
Epoch: 008/010 cost: 0.575154466
Training accuracy: 0.729
Test accuracy: 0.731
xgz
[[65796  7976]
 [24136 21659]]
(738132, 8)
data_combined:  (1194978, 8)
Epoch: 000/010 cost: 0.664872546
Training accuracy: 0.613
Epoch: 004/010 cost: 0.569548787
Training accuracy: 0.768
Epoch: 008/010 cost: 0.564347099
Training accuracy: 0.768
Test accuracy: 0.738
xes
[[61577 12129]
 [19142 26650]]
(738575, 8)
dat

Test accuracy: 0.713
xjl
[[62134 11762]
 [22558 23090]]
(736606, 8)
data_combined:  (1193452, 8)
Epoch: 000/010 cost: 0.665820431
Training accuracy: 0.617
Epoch: 004/010 cost: 0.584501249
Training accuracy: 0.723
Epoch: 008/010 cost: 0.582323150
Training accuracy: 0.739
Test accuracy: 0.725
xam
[[67657  5984]
 [26824 18881]]
(739729, 8)
data_combined:  (1196575, 8)
Epoch: 000/010 cost: 0.665497563
Training accuracy: 0.615
Epoch: 004/010 cost: 0.575251981
Training accuracy: 0.739
Epoch: 008/010 cost: 0.572094407
Training accuracy: 0.726
Test accuracy: 0.730
xhw
[[68751  5480]
 [26874 18553]]
(738518, 8)
data_combined:  (1195364, 8)
Epoch: 000/010 cost: 0.665218377
Training accuracy: 0.613
Epoch: 004/010 cost: 0.567944156
Training accuracy: 0.754
Epoch: 008/010 cost: 0.564220219
Training accuracy: 0.729
Test accuracy: 0.747
xey
[[65429  8712]
 [21527 23869]]
(738844, 8)
data_combined:  (1195690, 8)
Epoch: 000/010 cost: 0.665540614
Training accuracy: 0.609
Epoch: 004/010 cost: 0.572482087

data_combined:  (1193985, 8)
Epoch: 000/010 cost: 0.665455710
Training accuracy: 0.609
Epoch: 004/010 cost: 0.561010992
Training accuracy: 0.752
Epoch: 008/010 cost: 0.556840374
Training accuracy: 0.750
Test accuracy: 0.752
xbm
[[65309  8296]
 [21261 24533]]
(736064, 8)
data_combined:  (1192910, 8)
Epoch: 000/010 cost: 0.665669953
Training accuracy: 0.623
Epoch: 004/010 cost: 0.565504122
Training accuracy: 0.757
Epoch: 008/010 cost: 0.564529897
Training accuracy: 0.757
Test accuracy: 0.740
xcx
[[63341 10419]
 [20584 24947]]
(738678, 8)
data_combined:  (1195524, 8)
Epoch: 000/010 cost: 0.665466307
Training accuracy: 0.638
Epoch: 004/010 cost: 0.595231766
Training accuracy: 0.715
Epoch: 008/010 cost: 0.592084854
Training accuracy: 0.710
Test accuracy: 0.707
xjk
[[59394 14364]
 [20681 25114]]
(736960, 8)
data_combined:  (1193806, 8)
Epoch: 000/010 cost: 0.665682562
Training accuracy: 0.627
Epoch: 004/010 cost: 0.560827151
Training accuracy: 0.731
Epoch: 008/010 cost: 0.559584326
Training 

Epoch: 004/010 cost: 0.578424217
Training accuracy: 0.707
Epoch: 008/010 cost: 0.571814881
Training accuracy: 0.681
Test accuracy: 0.734
xfq
[[61990 11861]
 [19973 25692]]
(736486, 8)
data_combined:  (1193332, 8)
Epoch: 000/010 cost: 0.665633018
Training accuracy: 0.603
Epoch: 004/010 cost: 0.564409161
Training accuracy: 0.730
Epoch: 008/010 cost: 0.560185699
Training accuracy: 0.740
Test accuracy: 0.743
xbq
[[68460  5323]
 [25338 20213]]
(737248, 8)
data_combined:  (1194094, 8)
Epoch: 000/010 cost: 0.665627290
Training accuracy: 0.617
Epoch: 004/010 cost: 0.559387230
Training accuracy: 0.751
Epoch: 008/010 cost: 0.556778242
Training accuracy: 0.751
Test accuracy: 0.752
xbk
[[66770  6892]
 [22746 23002]]
(736392, 8)
data_combined:  (1193238, 8)
Epoch: 000/010 cost: 0.665288344
Training accuracy: 0.588
Epoch: 004/010 cost: 0.567961463
Training accuracy: 0.728
Epoch: 008/010 cost: 0.565400858
Training accuracy: 0.733
Test accuracy: 0.734
xcr
[[60964 12439]
 [19289 26632]]
(737047, 8)
dat

Test accuracy: 0.740
xfd
[[63035 10794]
 [20225 25391]]
(736963, 8)
data_combined:  (1193809, 8)
Epoch: 000/010 cost: 0.665790123
Training accuracy: 0.593
Epoch: 004/010 cost: 0.575433687
Training accuracy: 0.725
Epoch: 008/010 cost: 0.570091630
Training accuracy: 0.715
Test accuracy: 0.736
xaq
[[62859 10878]
 [20649 24995]]
(738231, 8)
data_combined:  (1195077, 8)
Epoch: 000/010 cost: 0.665489637
Training accuracy: 0.627
Epoch: 004/010 cost: 0.580966012
Training accuracy: 0.729
Epoch: 008/010 cost: 0.579506600
Training accuracy: 0.699
Test accuracy: 0.728
xea
[[63081 10676]
 [21862 23889]]
(738006, 8)
data_combined:  (1194852, 8)
Epoch: 000/010 cost: 0.665640824
Training accuracy: 0.622
Epoch: 004/010 cost: 0.571406336
Training accuracy: 0.757
Epoch: 008/010 cost: 0.565947086
Training accuracy: 0.755
Test accuracy: 0.718
xdu
[[56028 17799]
 [15920 29739]]
(736442, 8)
data_combined:  (1193288, 8)
Epoch: 000/010 cost: 0.665813325
Training accuracy: 0.610
Epoch: 004/010 cost: 0.569684170

data_combined:  (1193349, 8)
Epoch: 000/010 cost: 0.665818985
Training accuracy: 0.631
Epoch: 004/010 cost: 0.568780802
Training accuracy: 0.752
Epoch: 008/010 cost: 0.563880568
Training accuracy: 0.757
Test accuracy: 0.739
xcq
[[63757 10105]
 [21010 24463]]
(736645, 8)
data_combined:  (1193491, 8)
Epoch: 000/010 cost: 0.665535559
Training accuracy: 0.610
Epoch: 004/010 cost: 0.579371134
Training accuracy: 0.732
Epoch: 008/010 cost: 0.574791439
Training accuracy: 0.685
Test accuracy: 0.730
xdo
[[64625  8805]
 [23377 22543]]
(738447, 8)
data_combined:  (1195293, 8)
Epoch: 000/010 cost: 0.665239129
Training accuracy: 0.624
Epoch: 004/010 cost: 0.595344612
Training accuracy: 0.705
Epoch: 008/010 cost: 0.589842366
Training accuracy: 0.719
Test accuracy: 0.705
xij
[[58667 14976]
 [20242 25645]]
(738175, 8)
data_combined:  (1195021, 8)
Epoch: 000/010 cost: 0.665335458
Training accuracy: 0.617
Epoch: 004/010 cost: 0.569726740
Training accuracy: 0.713
Epoch: 008/010 cost: 0.567828942
Training 

Epoch: 004/010 cost: 0.578413195
Training accuracy: 0.734
Epoch: 008/010 cost: 0.576105858
Training accuracy: 0.736
Test accuracy: 0.728
xcf
[[64037  9764]
 [22688 22927]]
(738559, 8)
data_combined:  (1195405, 8)
Epoch: 000/010 cost: 0.665488970
Training accuracy: 0.608
Epoch: 004/010 cost: 0.596933901
Training accuracy: 0.714
Epoch: 008/010 cost: 0.593193253
Training accuracy: 0.716
Test accuracy: 0.700
xik
[[57956 15818]
 [20048 25719]]
(739223, 8)
data_combined:  (1196069, 8)
Epoch: 000/010 cost: 0.665451582
Training accuracy: 0.614
Epoch: 004/010 cost: 0.585745276
Training accuracy: 0.700
Epoch: 008/010 cost: 0.581822020
Training accuracy: 0.724
Test accuracy: 0.716
xie
[[68714  5419]
 [28541 16933]]
(738367, 8)
data_combined:  (1195213, 8)
Epoch: 000/010 cost: 0.664936483
Training accuracy: 0.614
Epoch: 004/010 cost: 0.565300003
Training accuracy: 0.733
Epoch: 008/010 cost: 0.565280313
Training accuracy: 0.739
Test accuracy: 0.743
xeu
[[68664  4804]
 [25882 20172]]
(738819, 8)
dat

Test accuracy: 0.730
xbd
[[58071 15656]
 [16532 29105]]
(737505, 8)
data_combined:  (1194351, 8)
Epoch: 000/010 cost: 0.665674332
Training accuracy: 0.613
Epoch: 004/010 cost: 0.574091303
Training accuracy: 0.721
Epoch: 008/010 cost: 0.573693115
Training accuracy: 0.727
Test accuracy: 0.732
xdr
[[67315  6399]
 [25570 20152]]
(737612, 8)
data_combined:  (1194458, 8)
Epoch: 000/010 cost: 0.665767947
Training accuracy: 0.629
Epoch: 004/010 cost: 0.577146310
Training accuracy: 0.730
Epoch: 008/010 cost: 0.574866602
Training accuracy: 0.724
Test accuracy: 0.715
xgq
[[69171  4526]
 [29571 16178]]
(736578, 8)
data_combined:  (1193424, 8)
Epoch: 000/010 cost: 0.665753699
Training accuracy: 0.626
Epoch: 004/010 cost: 0.581186780
Training accuracy: 0.708
Epoch: 008/010 cost: 0.581464573
Training accuracy: 0.705
Test accuracy: 0.717
xdl
[[68947  4703]
 [29095 16598]]
(735702, 8)
data_combined:  (1192548, 8)
Epoch: 000/010 cost: 0.665688457
Training accuracy: 0.627
Epoch: 004/010 cost: 0.571040408

In [5]:

csv1 = pd.read_csv('m1/final_test_xaa.csv', header=None)
csv1.iloc[:,1:2].head()

for item in os.listdir('m1/'):
    if item != 'final_test_xaa.csv':
        if item not in csv1.columns.values 
            csv2 = pd.read_csv('m1/'+item, header=None, engine='python')
            csv1[item] = csv2.iloc[:,1:2]
            print(csv1.shape)
    else:
        pass
csv1.head()
    

(18790469, 3)
(18790469, 4)
(18790469, 5)
(18790469, 6)
(18790469, 7)
(18790469, 8)
(18790469, 9)
(18790469, 10)
(18790469, 11)
(18790469, 12)
(18790469, 13)
(18790469, 14)
(18790469, 15)
(18790469, 16)
(18790469, 17)
(18790469, 18)
(18790469, 19)
(18790469, 20)
(18790469, 21)
(18790469, 22)
(18790469, 23)
(18790469, 24)
(18790469, 25)
(18790469, 26)
(18790469, 27)
(18790469, 28)
(18790469, 29)
(18790469, 30)
(18790469, 31)
(18790469, 32)
(18790469, 33)
(18790469, 34)
(18790469, 35)
(18790469, 36)
(18790469, 37)
(18790469, 38)
(18790469, 39)
(18790469, 40)
(18790469, 41)
(18790469, 42)
(18790469, 43)
(18790469, 44)
(18790469, 45)
(18790469, 46)
(18790469, 47)
(18790469, 48)
(18790469, 49)
(18790469, 50)
(18790469, 51)
(18790469, 52)
(18790469, 53)
(18790469, 54)
(18790469, 55)
(18790469, 56)
(18790469, 57)
(18790469, 58)
(18790469, 59)
(18790469, 60)
(18790469, 61)
(18790469, 62)
(18790469, 63)
(18790469, 64)
(18790469, 65)
(18790469, 66)
(18790469, 67)
(18790469, 68)
(18790469, 69)
(1

IsADirectoryError: [Errno 21] Is a directory: 'm1/.ipynb_checkpoints'

In [7]:
csv1.head()
csv1.to_csv('m1/final.csv')

In [15]:
count =0
for item in os.listdir('m1/'):
    if item != 'final_test_xaa.csv':
        if item not in csv1.columns.values : 
            print(item,count)
            csv2 = pd.read_csv('m1/'+item, header=None, engine='python')
            csv1[item] = csv2.iloc[:,1:2]
            print(csv1.shape)
    else:
        pass
csv1.head()

final_test_xdh.csv 0
(18790469, 183)
final_test_xfs.csv 0
(18790469, 184)
final_test_xbj.csv 0
(18790469, 185)
final_test_xij.csv 0
(18790469, 186)
final_test_xek.csv 0
(18790469, 187)
final_test_xbk.csv 0
(18790469, 188)
final_test_xgn.csv 0
(18790469, 189)
final_test_xet.csv 0
(18790469, 190)
final_test_xah.csv 0
(18790469, 191)
final_test_xgg.csv 0
(18790469, 192)
final_test_xfu.csv 0
(18790469, 193)
final_test_xcf.csv 0
(18790469, 194)
final_test_xih.csv 0
(18790469, 195)
final_test_xam.csv 0
(18790469, 196)
final_test_xia.csv 0
(18790469, 197)
final_test_xio.csv 0
(18790469, 198)
final_test_xcb.csv 0
(18790469, 199)
final_test_xgq.csv 0
(18790469, 200)
final_test_xac.csv 0
(18790469, 201)
final_test_xdo.csv 0
(18790469, 202)
final_test_xed.csv 0
(18790469, 203)
final_test_xcp.csv 0
(18790469, 204)
final_test_xcj.csv 0
(18790469, 205)
final_test_xdm.csv 0
(18790469, 206)
final_test_xbp.csv 0
(18790469, 207)
final_test_xhj.csv 0
(18790469, 208)
final_test_xiv.csv 0
(18790469, 209)
f

,0,1,final_test_xbl.csv,final_test_xcl.csv,final_test_xbv.csv,final_test_xct.csv,final_test_xgk.csv,final_test_xdk.csv,final_test_xhl.csv,final_test_xji.csv,...,final_test_xjo.csv,final_test_xbz.csv,final_test_xfi.csv,final_test_xcg.csv,final_test_xgj.csv,final_test_xjg.csv,final_test_xhu.csv,final_test_xga.csv,final_test_xdp.csv,final_test_xim.csv
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,1,1,0,0,1,0,0,...,1,1,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
#df.drop(df.columns[i], axis=1)
count = csv1.iloc[:,1:].sum(axis=1)
count[:5]

0     0
1     0
2    13
3    58
4     0
dtype: int64

In [18]:
len(count)

18790469

In [19]:
np.savetxt('data/count2.csv' , count)
csv1.to_csv('data/final2.csv')


In [ ]:
final_test['click_id'] = final_test['click_id'].astype(np.int32)
fa =  np.zeros(18790468)
for i in range(0,len(count)):
    if count[i] > 125 :
        fa[i] = 1
    else: 
        pass
        
a = np.vstack((final_test['click_id'].values,fa))
np.savetxt('./m1/final_test_.csv',a, fmt='%i', delimiter=",")

In [100]:
final_test['click_id'] = final_test['click_id'].astype(np.int32)
fa =  np.zeros(18790468)
for i in range(0,csv1.shape[0]):
    chk_score = sum(sum(csv1.iloc[i:i+1,:].values))
    if chk_score >= 91 :
        fa[i] = 1
    else: 
        pass
        
a = np.vstack((final_test['click_id'].values,fa))
np.savetxt('./m1/final_test_.csv',a, fmt='%i', delimiter=",")

KeyboardInterrupt: 

In [1]:
csv1.head()

NameError: name 'csv1' is not defined

In [2]:
# data is divided into positive negative samples and 7 gb file of negatives is further divided into 250 splits. 
# $ split -n l/250 negatives.csv
print('Negative samples')
data_neg = pd.read_csv('data/negatives/xaa', header=None)
print(data_neg.head())
print(data_neg.shape)
print('\n Positive samples')
data_pos = pd.read_csv('data/positives/positives.csv',header=None)
print(data_pos.head())
print(data_pos.shape)

Negative samples
        0   1  2   3    4                    5   6  7
0   83230   3  1  13  379  2017-11-06 14:32:21 NaN  0
1   17357   3  1  19  379  2017-11-06 14:33:34 NaN  0
2   35810   3  1  13  379  2017-11-06 14:34:12 NaN  0
3   45745  14  1  13  478  2017-11-06 14:34:52 NaN  0
4  161007   3  1  13  379  2017-11-06 14:35:08 NaN  0
(737196, 8)

 Positive samples
        0   1  2   3    4                    5                    6  7
0  204158  35  1  13   21  2017-11-06 15:41:07  2017-11-07 08:17:19  1
1   29692   9  1  22  215  2017-11-06 16:00:02  2017-11-07 10:05:22  1
2   64516  35  1  13   21  2017-11-06 16:00:02  2017-11-06 23:40:50  1
3  172429  35  1  46  274  2017-11-06 16:00:03  2017-11-07 00:55:29  1
4  199085  35  1  13  274  2017-11-06 16:00:04  2017-11-06 23:04:54  1
(456846, 8)


In [3]:
# Build x_train, y_tarin, x_test, y_test
data_combined =  pd.concat([data_pos, data_neg])
data_combined = data_combined.sample(frac=1)
print('data_combined: ', data_combined.shape)


data_combined:  (1194042, 8)


In [41]:
data_f = data_combined.iloc[:,0:6]

#data_f.astype(float)
labels = data_combined.iloc[:,7:8]
labels.astype(float)
type(labels)
#n_values = np.max(labels) 
labels_ = np.eye(2)[(labels.values).reshape(-1)]
labels_.shape
# labels = labels.values
# enc = OneHotEncoder()
# enc.fit(labels)
# labels= enc.transform(labels)
# labels.shape
#del data_combined, data_neg, data_pos
# y = np.zeros((data_combined.shape[0], 2))
# y[np.arange(data_combined.shape[0]), y] = 1

(1194042, 2)

In [61]:
from datetime import time
df = data_f
morning_start = time(6)
morning_end = time(12)
afternoon_start = time(13)
afternoon_end = time(17)
evening_start = time(18)
evening_end = time(22)
night_start= time(23)
night_end= time(5)
df.head()
df.iloc[:,5] = pd.to_datetime(df.iloc[:,5])
df['time_of_day'] = df.iloc[:,5].apply(lambda x: x.time())
#df['time_of_day'].head

df['morning'] = (df['time_of_day'] >= time(8,0)) & (df['time_of_day'] <= time(12,0))
df['afternoon'] = (df['time_of_day'] >= time(12,1)) & (df['time_of_day'] <= time(17,0))
df['evening'] = (df['time_of_day'] >= time(17,1)) & (df['time_of_day'] <= time(20,0))
df['night'] = (df['time_of_day'] >= time(20,1)) & (df['time_of_day'] <= time(23,59))
df['late_night'] = (df['time_of_day'] >= time(0,1)) & (df['time_of_day'] <= time(8,0))
df['night'] = df['night'].astype(float)
df['afternoon'] = df['afternoon'].astype(float)
df['late_night'] = df['late_night'].astype(float)
df['morning'] = df['morning'].astype(float)
df['evening'] = df['evening'].astype(float)
df.head()


,0,1,2,3,4,5,time_of_day,morning,afternoon,evening,night,late_night
392605,42424,9,1,17,466,2017-11-09 07:58:17,07:58:17,0.0,0.0,0.0,0.0,1.0
276471,45745,20,1,13,259,2017-11-08 12:08:02,12:08:02,0.0,1.0,0.0,0.0,0.0
258022,46542,14,1,13,489,2017-11-06 16:05:05,16:05:05,0.0,1.0,0.0,0.0,0.0
672377,131715,18,1,17,121,2017-11-06 16:14:00,16:14:00,0.0,1.0,0.0,0.0,0.0
327700,80975,14,1,53,463,2017-11-06 16:06:16,16:06:16,0.0,1.0,0.0,0.0,0.0


In [64]:
df2 = df.iloc[:,[0,1,2,3,4,7,8,9,10,11]]
df2.head()
df2.astype(float).head()
del df

,0,1,2,3,4,morning,afternoon,evening,night,late_night
392605,42424.0,9.0,1.0,17.0,466.0,0.0,0.0,0.0,0.0,1.0
276471,45745.0,20.0,1.0,13.0,259.0,0.0,1.0,0.0,0.0,0.0
258022,46542.0,14.0,1.0,13.0,489.0,0.0,1.0,0.0,0.0,0.0
672377,131715.0,18.0,1.0,17.0,121.0,0.0,1.0,0.0,0.0,0.0
327700,80975.0,14.0,1.0,53.0,463.0,0.0,1.0,0.0,0.0,0.0


In [65]:
x_train, x_test, y_train, y_test = train_test_split(df2,labels_,test_size = 0.1,random_state = 100)
x_train.astype(float)
x_test.astype(float)
y_train.astype(float)
y_test.astype(float)

array([[1., 0.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]])

In [80]:
final_test = pd.read_csv('./data/test/test.csv')
final_test.head()

,click_id,ip,app,device,os,channel,click_time
0,0,5744,9,1,3,107,2017-11-10 04:00:00
1,1,119901,9,1,3,466,2017-11-10 04:00:00
2,2,72287,21,1,19,128,2017-11-10 04:00:00
3,3,78477,15,1,13,111,2017-11-10 04:00:00
4,4,123080,12,1,13,328,2017-11-10 04:00:00


In [94]:
#del df
from datetime import time
df = final_test
morning_start = time(6)
morning_end = time(12)
afternoon_start = time(13)
afternoon_end = time(17)
evening_start = time(18)
evening_end = time(22)
night_start= time(23)
night_end= time(5)

df.iloc[:,6] = pd.to_datetime(df.iloc[:,6])
df['time_of_day'] = df.iloc[:,6].apply(lambda x: x.time())
#df['time_of_day'].head
df['morning'] = (df['time_of_day'] >= time(8,0)) & (df['time_of_day'] <= time(12,0))
df['afternoon'] = (df['time_of_day'] >= time(12,1)) & (df['time_of_day'] <= time(17,0))
df['evening'] = (df['time_of_day'] >= time(17,1)) & (df['time_of_day'] <= time(20,0))
df['night'] = (df['time_of_day'] >= time(20,1)) & (df['time_of_day'] <= time(23,59))
df['late_night'] = (df['time_of_day'] >= time(0,1)) & (df['time_of_day'] <= time(8,0))
df['night'] = df['night'].astype(float)
df['afternoon'] = df['afternoon'].astype(float)
df['late_night'] = df['late_night'].astype(float)
df['morning'] = df['morning'].astype(float)
df['evening'] = df['evening'].astype(float)
print(df.head())
test_df = df
#del df
test_df.head()

   click_id      ip  app  device  os  channel          click_time time_of_day  \
0         0    5744    9       1   3      107 2017-11-10 04:00:00    04:00:00   
1         1  119901    9       1   3      466 2017-11-10 04:00:00    04:00:00   
2         2   72287   21       1  19      128 2017-11-10 04:00:00    04:00:00   
3         3   78477   15       1  13      111 2017-11-10 04:00:00    04:00:00   
4         4  123080   12       1  13      328 2017-11-10 04:00:00    04:00:00   

   morning  afternoon  evening  night  late_night  
0      0.0        0.0      0.0    0.0         1.0  
1      0.0        0.0      0.0    0.0         1.0  
2      0.0        0.0      0.0    0.0         1.0  
3      0.0        0.0      0.0    0.0         1.0  
4      0.0        0.0      0.0    0.0         1.0  


,click_id,ip,app,device,os,channel,click_time,time_of_day,morning,afternoon,evening,night,late_night
0,0,5744,9,1,3,107,2017-11-10 04:00:00,04:00:00,0.0,0.0,0.0,0.0,1.0
1,1,119901,9,1,3,466,2017-11-10 04:00:00,04:00:00,0.0,0.0,0.0,0.0,1.0
2,2,72287,21,1,19,128,2017-11-10 04:00:00,04:00:00,0.0,0.0,0.0,0.0,1.0
3,3,78477,15,1,13,111,2017-11-10 04:00:00,04:00:00,0.0,0.0,0.0,0.0,1.0
4,4,123080,12,1,13,328,2017-11-10 04:00:00,04:00:00,0.0,0.0,0.0,0.0,1.0


In [100]:
test_df2 = test_df.iloc[:,[1,2,3,4,5,8,9,10,11,12]]
test_df2.head()

,ip,app,device,os,channel,morning,afternoon,evening,night,late_night
0,5744,9,1,3,107,0.0,0.0,0.0,0.0,1.0
1,119901,9,1,3,466,0.0,0.0,0.0,0.0,1.0
2,72287,21,1,19,128,0.0,0.0,0.0,0.0,1.0
3,78477,15,1,13,111,0.0,0.0,0.0,0.0,1.0
4,123080,12,1,13,328,0.0,0.0,0.0,0.0,1.0


In [95]:
del df

In [66]:
Y_LABEL = 'y'                                   			        # Name of the variable to be predicted
#KEYS = [i for i in raw_data.keys().tolist() if i != Y_LABEL]	# Name of predictors
N_INSTANCES = df2.shape[0]                     			    # Number of instances
N_INPUT = x_train.shape[1]                      			    # Input size
N_CLASSES = 2                                    			    # Number of classes (output size)
TEST_SIZE = 0.1                                    			      # Test set size (% of dataset)
TRAIN_SIZE = int(N_INSTANCES * (1 - TEST_SIZE))     			    # Train size
LEARNING_RATE = 0.001                               			    # Learning rate
TRAINING_EPOCHS =   1                             			    # Number of epochs
BATCH_SIZE = 100                                    			    # Batch size
DISPLAY_STEP = 1                                    			    # Display progress each x epochs
HIDDEN_SIZE = 10          	                                   		      # Number of hidden neurons 256
ACTIVATION_FUNCTION_OUT = tf.nn.sigmoid                          # Last layer act fct
STDDEV = 0.1                                        			    # Standard deviation (for weights random init)
RANDOM_STATE = 100   		

In [67]:
n_input = N_INPUT                   # input n labels
n_hidden_1 = 20            # 1st layer
n_hidden_2 = 40            # 2nd layer
n_hidden_3 = 30            # 3rd layer
n_hidden_4 = 10           # 4th layer
n_classes = N_CLASSES               # output m classes

In [68]:

X = tf.placeholder(tf.float32, [None, n_input],name='X')
y = tf.placeholder(tf.float32, [None, n_classes],name='y')
dropout_keep_prob = tf.placeholder(tf.float32)


weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1],stddev=STDDEV)),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2],stddev=STDDEV)),
    'h3': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_3],stddev=STDDEV)),
    'h4': tf.Variable(tf.random_normal([n_hidden_3, n_hidden_4],stddev=STDDEV)),
    'out': tf.Variable(tf.random_normal([n_hidden_4, n_classes],stddev=STDDEV)),                                   
}

biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'b3': tf.Variable(tf.random_normal([n_hidden_3])),
    'b4': tf.Variable(tf.random_normal([n_hidden_4])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}    
   
def mlp(_X, _weights, _biases, dropout_keep_prob):
    layer1 = tf.nn.dropout(tf.nn.relu(tf.add(tf.matmul(_X, _weights['h1']), _biases['b1'])), dropout_keep_prob)
    layer2 = tf.nn.dropout(tf.nn.relu(tf.add(tf.matmul(layer1, _weights['h2']), _biases['b2'])), dropout_keep_prob)
    layer3 = tf.nn.dropout(tf.nn.relu(tf.add(tf.matmul(layer2, _weights['h3']), _biases['b3'])), dropout_keep_prob)
    layer4 = tf.nn.dropout(tf.nn.relu(tf.add(tf.matmul(layer3, _weights['h4']), _biases['b4'])), dropout_keep_prob)
    out = ACTIVATION_FUNCTION_OUT(tf.add(tf.matmul(layer4, _weights['out']), _biases['out']))
    return out



In [69]:
pred = mlp(X, weights, biases, dropout_keep_prob)

# Loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y)) # softmax loss
optimizer = tf.train.AdamOptimizer(learning_rate = LEARNING_RATE).minimize(cost)

# Accuracy
correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
confusion_matrix = tf.confusion_matrix(labels= tf.argmax(y, 1),predictions = tf.argmax(pred, 1), num_classes=2)
predictions =  tf.argmax(pred,1)

In [101]:
tf.set_random_seed(1234)
saver = tf.train.Saver()
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    
    for epoch in range(TRAINING_EPOCHS):
        avg_cost = 0.
        total_batch = int(x_train.shape[0] / BATCH_SIZE)
        X_batches = np.array_split(x_train, total_batch)
        Y_batches = np.array_split(y_train, total_batch)
        # Loop over all batches
        for i in range(total_batch):
#             randidx = np.random.randint(int(TRAIN_SIZE), size = BATCH_SIZE)
#             batch_xs = x_tr[randidx, :]
#             batch_ys = labels[randidx, :]
            batch_xs, batch_ys =  X_batches[i], Y_batches[i]
            # Fit using batched data
            sess.run(optimizer, feed_dict={X: batch_xs, y: batch_ys, dropout_keep_prob: 0.8})
            # Calculate average cost
            avg_cost += sess.run(cost, feed_dict={X: batch_xs, y: batch_ys, dropout_keep_prob:1.})/total_batch
        # Display progress
        if epoch % DISPLAY_STEP == 0:
            print("Epoch: %03d/%03d cost: %.9f" % (epoch, TRAINING_EPOCHS, avg_cost))
            train_acc = sess.run(accuracy, feed_dict={X: batch_xs, y: batch_ys, dropout_keep_prob:1.})
            print("Training accuracy: %.3f" % (train_acc))
    test_acc = sess.run(accuracy, feed_dict={X: x_test, y: y_test, dropout_keep_prob:1.})
    confusion = sess.run(confusion_matrix, feed_dict={X: x_test, y: y_test, dropout_keep_prob:1.})
    print("Test accuracy: %.3f" % (test_acc))
    answer  = sess.run(predictions, feed_dict={X: x_test, y: y_test, dropout_keep_prob:1.})
    final_test_acc = sess.run(predictions, feed_dict={X: test_df2.values, y: y_test, dropout_keep_prob:1.})
    #print(answer)
    np.savetxt('./m1/final_test_acc', final_test_acc)
    np.savetxt('./m1/answer', answer)
    print(confusion)
    saver.save(sess, './m1/model/')


Epoch: 000/001 cost: 0.583111460
Training accuracy: 0.810
Test accuracy: 0.787
[[66803  6909]
 [18532 27161]]


In [103]:
temp = np.loadtxt('./m1/final_test_acc')
np.savetxt('./m1/final_test_acc.csv', temp, delimiter=",")

In [ ]:
temp.head()

In [151]:
temp = temp.astype(np.int32)
final_test['click_id'] = final_test['click_id'].astype(np.int32)
a = np.vstack((final_test['click_id'].values,temp))
a = a.T
a = np.int32(a)
#a[:5]
np.savetxt('./m1/final_test_acc1.csv', a, fmt='%i', delimiter=",")